In [31]:
import os
import pandas as pd
from langchain_openai import OpenAIEmbeddings # 임배딩 라이브러리
from langchain_openai import ChatOpenAI # OpenAI 라이브러리
from langchain_community.vectorstores import FAISS # 백터 데이터를 저장하고 유사도검색을 하는 라이브러리
from langchain_classic.chains import RetrievalQA # 기존 RetrievalQA가 lagnchain_classic으로 이동함
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
## 파일로드
filepath = "./data/car_prices.xlsx"
df = pd.read_excel(filepath)

## col을 모두 조합시켜서 설명하는 하나의 col생성
df["text"] = df.apply(lambda row: f"{row['제조사']} {row['모델']} 가격: {row['가격(만원)']}만원", axis=1)

## llm 모델 준비
llm = ChatOpenAI(model="gpt-4o-mini")

## embedding 모델 준비
embeddings = OpenAIEmbeddings()

## 임배딩해서 인덱스 저장
vectorstore = FAISS.from_texts(df["text"].tolist(), embeddings)

## "백터DB" 로컬 디스크 저장
faiss_db_path = "faiss_db_car_prices"
vectorstore.save_local(faiss_db_path)

print(df["text"].head(5))

0       혼다 시빅 가격: 3802000만원
1      현대 소나타 가격: 5485000만원
2    쉐보레 콜로라도 가격: 5343000만원
3    쉐보레 트래버스 가격: 5166000만원
4    포르쉐 911 가격: 29798000만원
Name: text, dtype: object


In [33]:
## 저장된 "백터DB"를 로드
faiss_db_path = "faiss_db_car_prices"
vectorstore = FAISS.load_local(faiss_db_path, embeddings, allow_dangerous_deserialization=True)

In [34]:
## 검색가능한 리트리벌 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [35]:
## 질의응답 시스템 생성
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

In [36]:
## 쿼리
query = "현대자동차에서 가장 비싼 자동차는 무엇인가?"
response = qa_chain.invoke(query)
print(response)

{'query': '현대자동차에서 가장 비싼 자동차는 무엇인가?', 'result': '저는 그에 대한 정보를 가지고 있지 않습니다.'}


In [37]:
## 쿼리
query = "아반떼의 제조사는 무엇인가"
response = qa_chain.invoke(query)
print(response)

{'query': '아반떼의 제조사는 무엇인가', 'result': '아반떼의 제조사는 현대자동차입니다.'}


In [38]:
## 쿼리
query = "1억 이상의 가격이 있는 자동차를 알려달라"
response = qa_chain.invoke(query)
print(response)

{'query': '1억 이상의 가격이 있는 자동차를 알려달라', 'result': '1억 이상의 가격이 있는 자동차는 맥라렌 720S입니다. 볼보 XC60은 가격이 1억 이하입니다.'}


In [39]:
## 쿼리
query = "볼보 XC60은 얼마야?"
response = qa_chain.invoke(query)
print(response)

{'query': '볼보 XC60은 얼마야?', 'result': '볼보 XC60의 가격은 11,571,000만원, 10,311,000만원, 그리고 5,507,000만원입니다.'}
